In [ ]:
from IPython.display import HTML

import pysal as ps
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import folium
import random as RD


In [ ]:

import folium
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

## Base layer for south

In [ ]:
import folium
map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5)
map_osm.geo_json(geo_path = 'data/south.json')
embed_map(map_osm)

## Binding Attribute Data to the Map

In [ ]:
import json

f = open(r'data/south.json')
q = json.load(f)
f.close()

In [ ]:
q.keys()

In [ ]:
features = q['features']
len(features)

In [ ]:
import pandas as pd
indices = []
values = []
for feature in features:
    indices.append(str(feature['properties']['FIPS']))
    values.append(feature['properties']['HR80'])

In [ ]:
df = pd.DataFrame({'HR80': values,
                   'FIPS': indices} )

In [ ]:
y = np.array(df.HR80.tolist())

In [ ]:
import numpy as np
import pysal as ps
w = ps.queen_from_shapefile('data/south.shp')
w.transform = 'r'

In [ ]:
y = np.array(df['HR80'].tolist())

## Local Moran
$li_i = \frac{n-1}{\sum_j z_j^2}z_i \sum_j w_{i,j} z_j$

In [ ]:
li_hr80 = ps.Moran_Local(y,w)

In [ ]:
li_hr80.p_sim.shape

In [ ]:
sig01 = 1.* (li_hr80.p_sim<=0.01)

In [ ]:
sig01.sum()

In [ ]:
sig05 = 1.* (li_hr80.p_sim<=0.05)

In [ ]:
sig05.sum()

In [ ]:
df['sig01']  = sig01
df['sig05'] = sig05

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','HR80'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80')
embed_map(map_osm)

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','sig01'],
                 fill_color='YlOrRd', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=[0,1],
                 legend_name='Homicide Rate HR80 - Significant LISA (0.01)')
embed_map(map_osm)

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','sig01'],
                 fill_color='YlOrRd', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80 - Significant LISA (0.01)')
embed_map(map_osm)

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','sig05'],
                 fill_color='YlOrRd', fill_opacity=0.7,
                 line_opacity=0.2,
                 legend_name='Homicide Rate HR80 - Significant LISA (0.05)')
embed_map(map_osm)

In [ ]:
li_hr80.q

In [ ]:
cold_spots = 1. * (li_hr80.q==3) * (li_hr80.p_sim < 0.01)

In [ ]:
cold_spots.sum()

In [ ]:
# Cold spots in pandas'y way
lisa_df = pd.DataFrame({'q': li_hr80.q, 'p_sim': li_hr80.p_sim})
cold = lisa_df[(lisa_df['q']==3) & (lisa_df['p_sim']<0.01)]
print cold.info()
print '\n~~~~~~~~~~\n'
# Hot spots in pandas'y way
hot = lisa_df[(lisa_df['q']==1) & (lisa_df['p_sim']<0.01)]
print hot.info()

In [ ]:
hot_spots = 1. * (li_hr80.q==1) * (li_hr80.p_sim < 0.01)

In [ ]:
hot_spots.sum()

In [ ]:
df['hot_01'] = hot_spots
df['cold_01'] = cold_spots

In [ ]:
df.hot_01

In [ ]:
df.cold_01

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','hot_01'],
                 fill_color='YlOrRd', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=[0,1],
                 legend_name='Homicide Rate HR80 - Hot Spots (0.01)')
embed_map(map_osm)

In [ ]:

map_osm = folium.Map(location=[33.7550, -87.3900], zoom_start=5,)
map_osm.geo_json(geo_path='data/south.json',
                 key_on='feature.properties.FIPS',
                 data_out='data.json', data=df,
                 columns=['FIPS','cold_01'],
                 fill_color='YlGnBu', fill_opacity=0.7,
                 line_opacity=0.2,
                 threshold_scale=[0,1],
                 legend_name='Homicide Rate HR80 - Cold Spots (0.01)')
embed_map(map_osm)

In [ ]:
dbf = ps.open('data/south.dbf')
dbf.header

In [ ]:
 df['STATE_NAME'] = dbf.by_col('STATE_NAME')

In [ ]:
df['County'] = dbf.by_col('NAME')

In [ ]:
hot_01 = df['hot_01']

In [ ]:
hot  = df[hot_01>0]

In [ ]:
hot

In [ ]:
df[['STATE_NAME', 'County']]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[[0,2,4]]

In [ ]:
df[df.hot_01 > 0 ].all()

In [ ]:
df[0:10]

In [ ]:
hot = df[df['hot_01']>0]

In [ ]:
hot

In [ ]:
df[df['hot_01']>0]

In [ ]:
df[df['cold_01']>0]

In [ ]:
print df[df['cold_01']>0].to_string() 

In [ ]:
hot_01

In [ ]:
y

In [ ]:
w = ps.queen_from_shapefile('data/south.shp')
w.transform = 'r'
wy = ps.lag_spatial(w,y)

In [ ]:
plt.scatter(y,wy, alpha=0.5, lw=0)
plt.vlines(y.mean(),wy.min(),wy.max())
plt.hlines(wy.mean(),y.min(),y.max())
plt.xlabel('y')
plt.ylabel('wy')
lm = ps.spreg.OLS(wy[:, None], y[:, None])
plt.plot(y, lm.predy, color='red')
plt.title('Moran Scatter Plot')
plt.show()

In [ ]:
c = 2*hot_01 + df.cold_01
plt.scatter(y,wy, c=c, s=(sig01+1)*50)
plt.vlines(y.mean(),wy.min(),wy.max())
plt.hlines(wy.mean(),y.min(),y.max())
plt.xlabel('y')
plt.ylabel('wy')
plt.title('Moran Scatter Plot')

In [ ]:
sig01.sum()

In [ ]:
df['wHR80'] = wy

In [ ]:
print df[df['cold_01']>0].to_string() 

In [ ]:
print df[df['hot_01']>0].to_string() 

In [ ]:
lg = ps.G_Local(y, w)

In [ ]:
lg.n

In [ ]:
sig = lg.p_sim < 0.01

In [ ]:
sig

In [ ]:
lg.p_sim[sig]

In [ ]:
lgs = ps.G_Local(y,w, star=True)

In [ ]:
lgs.p_sim[lgs.p_sim < 0.01]

In [ ]:
sum(sig)

In [ ]:
sum(lgs.p_sim < 0.01)